## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.xception.Xception(include_top=True, weights='imagenet',
                                     input_tensor=None, input_shape=None, 
                                     pooling=None, classes=1000)

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)

model.summary()


Using TensorFlow backend.


D:\Program Files\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.1)(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model深度： 179


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 170s 3ms/step - loss: 1.2530 - acc: 0.5821
Epoch 2/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.9797 - acc: 0.6694
Epoch 3/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.8616 - acc: 0.7039
Epoch 4/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.8095 - acc: 0.7188
Epoch 5/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.7760 - acc: 0.7329
Epoch 6/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.6728 - acc: 0.7705
Epoch 7/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.5730 - acc: 0.8016
Epoch 8/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.5192 - acc: 0.8209
Epoch 9/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.457

50000/50000 [==============================] - 159s 3ms/step - loss: 0.0645 - acc: 0.9802
Epoch 71/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0663 - acc: 0.9794
Epoch 72/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0628 - acc: 0.9807
Epoch 73/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0666 - acc: 0.9792
Epoch 74/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0639 - acc: 0.9804
Epoch 75/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0602 - acc: 0.9814
Epoch 76/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0605 - acc: 0.9819
Epoch 77/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0552 - acc: 0.9825
Epoch 78/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.0613 - acc: 0.9818
Epoch 79/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.05